In [103]:
import numpy as np
import pandas as pd
import matplotlib

In [104]:
data_path = 'D:/kagglehub/kagglehub/datasets/ai4a-lab/nanofluid-density-prediction/versions/1/Density_Prediction_Dataset.csv'
df = pd.read_csv(data_path, encoding='UTF-8-SIG')

print('DataFrame Shape:', df.shape)
df.head()

DataFrame Shape: (436, 10)


,Nano Particle,Base Fluid,Temperature (°C),Volume Concentration (ϕ),Density of Nano Particle 1 (ρnp),Density of Nano Particle 2 (ρnp),Density of Base Fluid (ρbf),Volume Mixture of Particle 1,Volume Mixture of Particle 2,Density (ρ)
0,Al₂O₃/SiO₂,Water,20,0.05,3890,2220,998.29,20,80,1004.814815
1,Al₂O₃/SiO₂,Water,30,0.05,3890,2220,995.71,20,80,1002.222222
2,Al₂O₃/SiO₂,Water,40,0.05,3890,2220,992.25,20,80,997.037037
3,Al₂O₃/SiO₂,Water,50,0.05,3890,2220,988.02,20,80,994.444444
4,Al₂O₃/SiO₂,Water,60,0.05,3890,2220,983.13,20,80,990.000000


In [105]:
df[df.columns[3:]].corr();

In [106]:
# Проверка на дубликаты
duplicates = df.duplicated().sum()

# Удаление дубликатов (если они есть)
if duplicates > 0:
    df = df.drop_duplicates()

# Проверка на пропущенные значения
missing_values = df.isnull().sum()

# Удаление строк с пропущенными значениями
initial_rows = df.shape[0]
df = df.dropna()
final_rows = df.shape[0]
removed_rows = initial_rows - final_rows

In [107]:
data = df.values.tolist()

features = [row[:-1] for row in data]

labels = [row[-1] for row in data]
labels_origin = labels.copy()

In [108]:
import pandas as pd

# Извлечем первые два столбца (категориальные признаки)
cat_col1 = [item[0] for item in features] 
cat_col2 = [item[1] for item in features]  

# Остальные признаки (числовые)
remaining_features = [item[2:] for item in features]

In [109]:
# Создаем временный DataFrame для one-hot кодирования
df_cat = pd.DataFrame({
    'cat1': cat_col1,
    'cat2': cat_col2
})

df_cat_encoded = pd.get_dummies(df_cat, prefix=['cat1', 'cat2'], dtype=int)
cat_encoded = df_cat_encoded.values.tolist()

In [110]:
# Объединяем закодированные категориальные признаки с числовыми
features_processed = [
    cat_encoded[i] + remaining_features[i] 
    for i in range(len(features))
]


In [111]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

numeric_data = np.array([row[18:] for row in features_processed], dtype=float)

# Инициализируем MinMaxScaler, устанавливаем границы (-1, 1)
scaler = MinMaxScaler(feature_range=(-1, 1))

labels = np.array(labels)

numeric_data_norm = scaler.fit_transform(numeric_data)
labels = scaler.fit_transform(labels.reshape(-1,1))
# Собираем обратно полный набор признаков
features_normalized = []
for i, row in enumerate(features_processed):
    # Сохраняем первые 18 категориальных признаков без изменений
    new_row = list(row[:18])
    new_row.extend(numeric_data_norm[i].tolist())
    features_normalized.append(new_row)

In [112]:
features_normalized = [lst[:-1] for lst in features_normalized]
features_processed = [row[:-1] for row in features_processed]

In [113]:
features_normalized = np.array(features_normalized)
features_processed= np.array(features_processed)
labels = np.array(labels)
labels_origin = np.array(labels_origin)